In [1]:
%run "H:\Python\connect to database.ipynb"

In [312]:
import numpy as np
import pandas as pd
import re

In [359]:
#set parameter
hrexdt= '20111025'
ooiexdt = '201110'
dtfilter ='01-OCT-11'

In [ ]:
#set parameter
hrexdt= '201210'
ooiexdt = '201210'
dtfilter ='01-OCT-12'

In [ ]:
#set parameter
hrexdt= '20131018'
ooiexdt = '20131018'
dtfilter ='01-OCT-13'

In [492]:
#primary assignments with jobs poistion pay info
#inner join with cx, job,base pay
def primds(hrexdt, ooiexdt, dtfilter):
    PRIM =  pd.read_sql("select * from HR_"+hrexdt+".HR_PA_EMP_PRIMARY_ASSIGNMENT_V \
                   where '"+dtfilter+"' between start_date and end_date", EDW)
    PSV = pd.read_sql("select * from HR_"+hrexdt+".HR_PA_PERSON_SV \
                   where '"+dtfilter+"' between start_date and end_date", EDW)
    POS = pd.read_sql("select * from HR_"+hrexdt+".HR_OM_POSITIONS_V \
                   where '"+dtfilter+"' between start_date and end_date", EDW)
    JOB = pd.read_sql("select * from HR_"+hrexdt+".HR_OM_JOB_V \
                   where '"+dtfilter+"' between start_date and end_date", EDW)
    ACTION = pd.read_sql("select * from HR_"+hrexdt+".HR_PA_ACTIONS_V \
                   where '"+dtfilter+"' between start_date and end_date and EMP_STATUS_CD in ('1','3') ", EDW)

    CX = pd.read_sql("select * from HR_"+hrexdt+".HR_PA_PERNR_CROSSWALK_V ", EDW)
    DSPEC = pd.read_sql("select * from HR_"+hrexdt+".HR_PA_DATE_SPECIFICATIONS_SV \
                   where '"+dtfilter+"' between start_date and end_date", EDW)

    PDETAIL = pd.read_sql("select * from HR_"+hrexdt+".HR_PA_PERSON_DETAIL_SV \
                   where '"+dtfilter+"' between start_date and end_date", EDW)
    RES = pd.read_sql("select * from HR_"+hrexdt+".HR_PA_RESIDENCE_STATUS_SV \
                   where '"+dtfilter+"' between start_date and end_date", EDW)

    BN = pd.read_sql("select * from HR_"+hrexdt+".HR_BN_BNFT_SVC_INFO_V \
                   where '"+dtfilter+"' between start_date and end_date", EDW)
    ED = pd.read_sql("select * from HR_"+hrexdt+".HR_PA_PERSON_EDUCATION_V", EDW)

    BASEPAY = pd.read_sql("select * from HR_"+hrexdt+".HR_PA_BASE_PAY_V \
                   where '"+dtfilter+"' between base_pay_start_date and base_pay_end_date and SRC_STYPE = '0'", EDW)

    FASDETAIL = pd.read_sql("select * from HR_"+hrexdt+".HR_PA_FAS_DETAILS_V \
                   where '"+dtfilter+"' between start_date and end_date", EDW)
    
    Newhire = pd.read_sql("select * from HR_"+hrexdt+".HR_PA_ACTIONS_V \
                   where ACTN_TYP_CD='ZA' and   EMP_STATUS_CD in ('1','3') and "+\
                     "start_date between '01-JULY-"+dtfilter[7:]+"' and '30-SEP-"+dtfilter[7:]+"'"  , EDW)

    var = [x for x in PDETAIL.columns if re.match(r'racial_cat_cd_\d+',x) or x=='ethnc_cd']
    PDETAIL= PDETAIL.set_index('pers_id').filter(items=var)

    ActivePerson = ACTION[['pers_nbr']].drop_duplicates()
    ActivePerson['Active']='Y'

    EDPrsn = ED.groupby('pers_id')['cert_cd'].max().reset_index()
    Newhire = Newhire[['pers_nbr']].drop_duplicates()
    Newhire['NewHire']='Y'
#EDPrsn.head()

    return PRIM.drop(['curr_ind', 'src_last_updated_by', 'src_last_update_date', 'created_by',\
                   'create_date', 'last_updated_by', 'last_update_date'], axis=1).\
       join(CX.set_index('pers_nbr').loc[:,['pers_id']], on='pers_nbr', how='inner').\
       join(JOB.set_index('job_cd').loc[:,['job_ttl','eeo_cd','eeo_nm','emp_cat_cd','emp_cat_nm','fac_rnk_cd']], on='job_cd', how='inner').\
        join(PSV.set_index('pers_id').loc[:,['ssn','uuid','fst_nm','mid_nm', 'lst_nm','birth_date', 'gndr_cd', 'gndr_nm','zpid']], on='pers_id').\
        join(PDETAIL, on='pers_id').\
        join(RES.set_index('pers_id').loc[:,['res_status_cd','res_status_nm']] , on='pers_id').\
        join(DSPEC.set_index('pers_nbr').loc[:,['cont_svc_date']], on='pers_nbr').\
        join(ActivePerson.set_index('pers_nbr'), on='pers_nbr').\
        join(BN.set_index('pers_nbr').loc[:,['acum_fte_svc_mths','vstd_date']], on='pers_nbr').\
        join(EDPrsn.set_index('pers_id'), on='pers_id').\
        join(BASEPAY.set_index('pers_nbr').loc[:,['pay_scale_typ_cd','pay_scale_area_cd','wage_typ_cd_2','pymt_wage_typ_amt_2',\
                                                  'pay_scale_grp_cd','pay_scale_lvl_cd','capc_util_lvl', 'anl_sal']], on='pers_nbr', how='inner').\
        join(POS.set_index('pos_nbr').loc[:,['pos_ttl']], on='pos_nbr').\
        join(FASDETAIL.set_index('pers_id').loc[:,['status_cd', 'status_nm','tenure_cont_grant_dept','tenure_cont_status_grant_date',\
                                                   'tenure_exmp_agr_date','tenure_cont_sys_entr_date','promo_to_cur_rnk_entr_date']], on='pers_id').\
        join(Newhire.set_index('pers_nbr'), on='pers_nbr')
        


In [493]:
prim = primds(hrexdt='20111025', ooiexdt='201110', dtfilter='01-OCT-11')

C:\Users\chendi4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:36: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
C:\Users\chendi4\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1367: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [363]:
prim.shape

(63195, 72)

In [364]:
#filter only academic staff or support staff or GA
#remove non-employee/no-pay
#remove non-active
#remove PRIM_ASGN_END_DATE before frozen date
prim = prim[ prim['emp_cat_cd'].str.slice(0,1).isin(['F','S','G'])].query('emp_grp_cd != "5"').query('Active=="Y"')
prim['cutdt']= np.datetime64(pd.to_datetime(dtfilter))
prim = prim[(prim['prim_asgn_end_date'] == np.datetime64('1900-01-01')) | ( prim['prim_asgn_end_date'] >=  prim['cutdt'])]
 
prim['emp_cat_cd_1']= prim['emp_cat_cd'].str.slice(0,1)

In [365]:
#2011 job table do not have fac_rnk_cd, code fac_rnk_cd based on job_ttl 
if prim['fac_rnk_cd'].isnull().sum() < len(prim):
    prim['fac_rnk_cd'] = np.where(prim['fac_rnk_cd'].isnull(), np.nan,  prim['fac_rnk_cd'].str.slice(3,4).astype(int))
else:
    prim['fac_rnk_cd']= np.where(prim['job_ttl'].str.contains('professorial',case=False), np.nan,
                                np.where(prim['job_ttl'].str.contains('ASSISTANT INSTRUCTOR', case=False),  np.nan,
                                   np.where( prim['job_ttl'].str.contains('INSTRUCTOR' , case=False), 1,
                                       np.where(prim['job_ttl'].str.contains('ASSISTANT PROFESSOR',case=False),2,
                                           np.where(prim['job_ttl'].str.contains('ASSOCIATE PROFESSOR',case=False),3,
                                                   np.where(prim['job_ttl'].str.contains('PROF,',case=False),4, 
                                                           np.where(prim['job_ttl'].str.upper().str.endswith('PROF'),4,
                                                                   np.where(prim['job_ttl'].str.contains('PROFESSOR',case=False),4,
                                                                           np.where(prim['job_ttl'].str.contains('PROF-',case=False),4,np.nan))))))))
    )
    

In [366]:
prim.shape

(15161, 74)

In [367]:
#prim[['job_ttl','fac_rnk_cd']].query('fac_rnk_cd==fac_rnk_cd').drop_duplicates()

In [368]:
#total FTE by person and empgroup (F, G, S)
CUL= prim[['pers_id','pers_nbr','emp_cat_cd','capc_util_lvl','asgn_typ_cd','emp_sgrp_cd','emp_cat_cd_1']].\
dropna(subset=['pers_id', 'capc_util_lvl','emp_cat_cd'])


CULag = CUL.groupby(['pers_id','emp_cat_cd_1'])['capc_util_lvl'].sum().reset_index().\
rename(columns={'capc_util_lvl':'cul'})
CULag.head()

,pers_id,emp_cat_cd_1,cul
0,00001003,S,100.0
1,00001004,F,100.0
2,00001005,F,100.0
3,00001006,F,100.0
4,00001007,S,100.0


In [369]:
prim = prim.join(CULag.set_index(['pers_id','emp_cat_cd_1']), on=['pers_id','emp_cat_cd_1'] )

In [370]:
prim.shape

(15161, 75)

In [371]:
#from IPython.display import display

#pd.options.display.max_columns = None
#display(prim11.query('pers_nbr=="7352"'))

In [372]:
##########DerivedRank from addtil asginment#########
def adtrank(hrexdt, ooiexdt, dtfilter):
    ASGN = pd.read_sql("select * from HR_"+hrexdt+".hr_pa_fas_assignment_v \
                   where '"+dtfilter+"' between start_date and end_date", EDW)
    asgncol  =[x for x in ASGN.columns if not re.match(r'prim_asgn_also_rpt_to_\d',x) and not re.match(r'\w+_org_cd_\d+',x) ]
    asgndf = ASGN.filter(items=asgncol).set_index(['pers_nbr','start_date','end_date']).\
            filter(regex= r'\w+_\d')
    asgndfL = asgndf.stack().reset_index().rename(columns={'level_3':'variable',0:'value'})
    asgndfL['seq']= asgndfL['variable'].str.extract(r'\w+_(\d)$') 
    asgndfL['grp']= asgndfL['variable'].str.extract(r'(\w+)_\d$')
    asgndfL =  asgndfL.drop('variable',axis=1).set_index(['pers_nbr','start_date','end_date','seq','grp']).unstack()
    asgndfL.columns=asgndfL.columns.droplevel()
    asgndfL= asgndfL.reset_index()

#4 is professor 1 is instructor
#if fas_rnk_cd exist use fac_rnk_cd, if not use fas_asgn_nm

    if 'fas_rnk_cd' in asgndfL.columns:
        asgndfL['DerivedRnk']=  asgndfL['fas_rnk_cd'].str.slice(3,4).astype(int)
    elif 'fas_asgn_nm' in asgndfL.columns:
        asgndfL['DerivedRnk']= np.where(asgndfL['fas_asgn_nm'].str.contains('PROFESSORIAL'), np.nan,
                                 np.where(asgndfL['fas_asgn_nm'].str.contains('ASSISTANT INSTRUCTOR'),  np.nan,
                                   np.where( asgndfL['fas_asgn_nm'].str.contains('INSTRUCTOR'), 1,
                                       np.where(asgndfL['fas_asgn_nm'].str.contains('ASSISTANT PROFESSOR'),2,
                                           np.where(asgndfL['fas_asgn_nm'].str.contains('ASSOCIATE PROFESSOR'),3,
                                                   np.where(asgndfL['fas_asgn_nm'].str.contains('PROFESSOR'),4, 
                                                           np.where(asgndfL['fas_asgn_nm'].str.startswith('PROF'),4, np.nan)))))))
    else:
        print('no addtl rank field')

    #BC Derived Rank, exclude certain asgn_type and add lecturer
    asgndfL['DerivedRnk_BC']= np.where(asgndfL['asgn_typ_cd'].isin(['2','4','8','3','9','A','B']), np.nan,
                                  np.where( asgndfL['DerivedRnk'].isnull()==False, asgndfL['DerivedRnk'] ,
                                          np.where(asgndfL['fas_asgn_nm'].str.contains('LECTURER'), 0, np.nan)))
    return asgndfL

In [373]:
DerRank = adtrank(hrexdt='20111025', ooiexdt='201110', dtfilter='01-OCT-11').groupby('pers_nbr')['DerivedRnk'].max().reset_index()

DerRankBC = adtrank(hrexdt='20111025', ooiexdt='201110', dtfilter='01-OCT-11').groupby('pers_nbr')['DerivedRnk_BC'].max().reset_index()

DerRank= DerRank.join(DerRankBC.set_index('pers_nbr'), on='pers_nbr')

#merge with prim
prim = prim.join(DerRank.set_index(['pers_nbr']), on=['pers_nbr'] )

C:\Users\chendi4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  import sys
C:\Users\chendi4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  


In [374]:
prim.shape

(15161, 77)

In [375]:
#get HR org hirachy from OOI_OOI_ORG_STRUCT_CODE_AV
ORG_V = pd.read_sql('select * from OOI_'+ooiexdt+'.OOI_OOI_ORG_V', EDW)
STRUCT = pd.read_sql('select * from OOI_'+ooiexdt+'.OOI_OOI_ORG_STRUCT_CODE_AV', EDW)
STRUCTHR = STRUCT.query('struct_type=="HR"').\
join(ORG_V.query('org_type=="MAU"').set_index('org_ref_code').loc[:,['org_full_name']], rsuffix='_mau', on='mau').\
join(ORG_V.query('org_type=="DP"').set_index('org_ref_code').loc[:,['org_full_name']], rsuffix='_dept', on='dept').\
loc[:,['org_code','org_full_name','org_type','mau','org_full_name_mau','dept','org_full_name_dept']]
STRUCTHR['type']=2

#get HR org hirachy from ooi_ooi_org_struct_cd_lvl_rv
OOIV = pd.read_sql('select * from OOI_'+ooiexdt+'.ooi_ooi_org_struct_cd_lvl_rv ', EDW)
OOIHR = OOIV.query('structure_type=="HR"').loc[:,['org_code','org_full_name', 'org_type', 'mau_code','mau_name','dept_code','dept_name']]
OOIHR['type']=1

In [376]:
#put 2 sources together and use ooi_ooi_org_struct_cd_lvl_rv answer first if mutliple
STRUCTHR.columns = OOIHR.columns
HROOI = pd.concat([OOIHR,STRUCTHR  ])
HROOI['r']=HROOI.groupby('org_code')['type'].transform(min)
HROOI = HROOI.query('r==type').drop(['type','r'], axis=1)

In [377]:
prim = prim.join(HROOI.set_index('org_code').\
                     loc[:,['org_full_name','mau_code','mau_name','dept_code','dept_name']], on='org_cd')

In [378]:
prim['derived_rank']= prim[['fac_rnk_cd','DerivedRnk']].apply(np.nanmax, axis=1)
#hard code derived rank for certain job_cd
if dtfilter.endswith('11'):
    prim['derived_rank'] = np.where(prim['job_cd'].isin(['20001600','20001601']),4,prim['derived_rank'])

C:\Users\chendi4\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:4933: RuntimeWarning: All-NaN axis encountered
  labels=labels)


In [379]:
#prim['org_full_name'].isnull().sum()
prim.shape

(15161, 83)

In [380]:
##Cost distribution pay distributed by each pay org

In [381]:
FM = pd.read_sql("select * from HR_"+hrexdt+".HR_PY_FUND_MASTER_V \
                   where '"+dtfilter+"' between start_date and end_date", EDW)
CD = pd.read_sql("select * from HR_"+hrexdt+".HR_PA_COST_DISTRIBUTION_V \
                   where '"+dtfilter+"' between start_date and end_date and LBR_DSTR_TBL_TYP='DIST' ", EDW)
BASEPAY = pd.read_sql("select * from HR_"+hrexdt+".HR_PA_BASE_PAY_V \
                   where '"+dtfilter+"' between base_pay_start_date and base_pay_end_date and SRC_STYPE = '0'", EDW)


In [382]:
COSTDist = FM.loc[:,['org_cd','acct_pub_nm','fund_grp_cd','fund']].\
            join(CD.set_index('fund').loc[:,['pers_nbr','wt_pct','pymt_wage_typ_amt']], on='fund' , how='inner').\
            join(BASEPAY[['pers_nbr']].set_index('pers_nbr'), on='pers_nbr', how='inner').reset_index().\
            groupby(['pers_nbr','org_cd','fund_grp_cd','fund'])['wt_pct','pymt_wage_typ_amt'].sum()
COSTDist['sum_pymt_wage_typ_amt']=COSTDist.groupby(['pers_nbr'])['pymt_wage_typ_amt'].transform(sum)        
COSTDist = COSTDist.reset_index().join(prim.set_index('pers_nbr').loc[:,['pers_id']], on='pers_nbr', how='inner')

In [383]:
COSTDist.shape

(18966, 8)

In [384]:
COSTDist = COSTDist.join(HROOI.set_index('org_code').\
                     loc[:,['org_full_name','mau_code','mau_name','dept_code','dept_name']], on='org_cd')

In [385]:
COSTDist['Weight_Percent']= COSTDist['pymt_wage_typ_amt']/COSTDist['sum_pymt_wage_typ_amt']*100

In [386]:
###########################################################################################
##########following is for coding HR_HighLevel_FTE_by_Fund_cuc & HR_nacad_FTE_by_Fund_cuc

In [387]:
EmpFTE= prim.join(COSTDist.set_index(['pers_id','pers_nbr']), on=['pers_id','pers_nbr'], rsuffix='_pay', how='inner') 

In [388]:
EmpFTE['FTE_Funding_Derived']= EmpFTE['wt_pct'] * EmpFTE['capc_util_lvl']/10000
#EmpFTE['derived_rank']= EmpFTE[['fac_rnk_cd','DerivedRnk']].apply(np.nanmax, axis=1)

In [389]:
#EmpFTE[['fac_rnk_cd','DerivedRnk','derived_rank','job_cd']].query('job_cd=="20001600"').head(10)
#Statc
EmpFTE['Statc']= np.where( (EmpFTE['status_cd'].str.startswith('C')) & (EmpFTE['status_cd'].str.endswith('E')==False ),'CX',
                         np.where(EmpFTE['status_cd']=='TENR','FW',
                                 np.where(EmpFTE['status_cd']=='TPRO','FP',
                                         np.where(EmpFTE['derived_rank'].isnull(),'TS','TF'))))

EmpFTE['Employee_category']= np.where(EmpFTE['emp_cat_cd_1']=='G','Grad Asst',
                                     np.where(EmpFTE['emp_cat_cd_1']=='S','Non Acad',
                                            np.where(EmpFTE['Statc'].isin(['FW','FP']),'Tenure Sys',
                                                    np.where(EmpFTE['Statc']=='CX','Cont Staff',
                                                            np.where(EmpFTE['Statc']=='TS','Fix Staff',
                                                                    np.where(EmpFTE['Statc']=='TF','Fix Fac', np.nan)))) ))

EmpFTE['egrp']= np.where(EmpFTE['Employee_category']=='Grad Asst','ga',
                        np.where(EmpFTE['Employee_category']== 'Tenure Sys','ts',
                                np.where(EmpFTE['Employee_category']=='Fix Fac','or',
                                        np.where(EmpFTE['Employee_category']=='Fix Staff','fts',
                                                np.where(EmpFTE['Employee_category']=='Cont Staff','cs','fy')))))

EmpFTE['GFFTE']= np.where(EmpFTE['fund_grp_cd']=='GF', EmpFTE['FTE_Funding_Derived'],0)
EmpFTE['GFFTE']= np.where(EmpFTE['Employee_category'] != 'Non Acad', EmpFTE['GFFTE'],
                         np.where( (EmpFTE['fund']=='MSXC023271') & (EmpFTE['org_cd']=='10070386'), EmpFTE['FTE_Funding_Derived']*0.5,
                             np.where(EmpFTE['fund']=='MSXC023271', EmpFTE['FTE_Funding_Derived']*0.7, EmpFTE['GFFTE'])))

In [390]:
EmpFTEagg = EmpFTE.groupby(['pers_id','pers_nbr','mau_code_pay','dept_code_pay','egrp',\
                           'Employee_category','FTE_Funding_Derived','fund_grp_cd'])['FTE_Funding_Derived','GFFTE'].sum()
EmpFTEagg.columns = ['affte','gffte']
EmpFTEagg['ngfte']= EmpFTEagg['affte']- EmpFTEagg['gffte']
EmpFTEagg= EmpFTEagg.reset_index().\
melt(id_vars=['pers_id','pers_nbr','mau_code_pay','dept_code_pay','egrp','Employee_category','FTE_Funding_Derived','fund_grp_cd'], var_name='type', value_name='fte')
EmpFTEagg['vargrp']= EmpFTEagg['egrp']+EmpFTEagg['type']
EmpFTEagg = EmpFTEagg.pivot_table(index=['pers_id','pers_nbr','mau_code_pay','dept_code_pay',\
                                         'Employee_category','FTE_Funding_Derived','fund_grp_cd'], columns='vargrp', fill_value=0 )
EmpFTEagg.columns = EmpFTEagg.columns.droplevel()
EmpFTEagg['onraffte']= EmpFTEagg['csaffte'] + EmpFTEagg['ftsaffte']
EmpFTEagg['onrgffte']= EmpFTEagg['csgffte'] + EmpFTEagg['ftsgffte']
EmpFTEagg['onrngffte']= EmpFTEagg['csngfte'] + EmpFTEagg['ftsngfte']
EmpFTEagg = EmpFTEagg.reset_index()
EmpFTEagg['CUC'] = EmpFTEagg['mau_code_pay'] +  EmpFTEagg['dept_code_pay']
EmpFTEagg['NCUC'] = EmpFTEagg['CUC']+'00'
EmpFTEagg['CYear'] = '20'+ dtfilter[7:]
EmpFTEagg = EmpFTEagg.drop(['mau_code_pay','dept_code_pay'], axis=1).\
rename(columns={'fund_grp_cd':'Fund_Group', 'pers_id':'Person_ID','pers_nbr':'Personnel_Number'})

In [391]:
#EmpFTE.groupby(['pers_id','pers_nbr','mau_code_pay','dept_code_pay','egrp','Employee_category','FTE_Funding_Derived','fund_grp_cd'])\
#['FTE_Funding_Derived','GFFTE'].sum().head()
EmpFTE.query('pers_nbr=="1346"').loc[:,['pers_nbr','fund','fund_grp_cd','mau_code_pay','dept_code_pay','org_cd']]
#.loc[:,['Personnel_Number','fyaffte','fygffte','fyngfte','F']]

,pers_nbr,fund,fund_grp_cd,mau_code_pay,dept_code_pay,org_cd
60107,1346,MSXC023271,XF,72,654,10072386


In [392]:
EmpFTEagg.\
groupby('CYear')['gaaffte','gagffte', 'tsaffte','tsgffte',\
                 'oraffte','orgffte','onraffte','onrgffte','csaffte','csgffte','ftsaffte','ftsgffte','fyaffte','fygffte','fyngfte'].sum()


vargrp,gaaffte,gagffte,tsaffte,tsgffte,oraffte,orgffte,onraffte,onrgffte,csaffte,csgffte,ftsaffte,ftsgffte,fyaffte,fygffte,fyngfte
CYear,,,,,,,,,,,,,,,
2011,1537.6199,906.55645,1952.77712,1689.62048,885.6474,622.9896,1800.8428,732.6695,696.1397,339.6811,1104.7031,392.9884,6292.1537,3108.39841,3183.75529


In [393]:
##############################################
#following is to generate Acad ##########

In [394]:
Acad = prim[prim['emp_sgrp_cd'].isin(['AT','AS','B4','AC','A1','AW','AD','AX','B7','B8'])==False].query('emp_cat_cd_1=="F"').\
query('(asgn_typ_cd !=asgn_typ_cd) | (asgn_typ_cd != "4")')

In [395]:
Acad['STATC']= np.where(Acad['status_cd'].isin(['TENR','TPRO']),'FW',
                       np.where( (Acad['derived_rank'].notnull()) & (Acad['status_cd'].isnull()) , 'FT',
                                np.where( (Acad['derived_rank'].notnull() & (Acad['status_cd'].str.endswith('E')))  , 'FT',
                                        np.where(Acad['status_cd'].isin(['CLIB','CPLB']),'LW',
                                           np.where(Acad['emp_cat_cd']=='FLF','LT',
                                               np.where(Acad['status_cd'].isin(['CEXT','CPEX']), 'MW',
                                                       np.where(Acad['emp_cat_cd']=='FEF','MT',
                                                               np.where(Acad['status_cd'].isin(['CNSC','CPNS']), 'NW',
                                                                       np.where(Acad['emp_cat_cd']=='FNF','NT',
                                                                               np.where(Acad['status_cd'].isin(['CPSP','CSPC']), 'SW',
                                                                                       np.where(Acad['emp_cat_cd']=='FSF','ST',
                                                                                               np.where(Acad['status_cd']=='COTR','UW','UT' ))))))))))
                              
                               ))

Acad['FULLPART']= np.where(Acad['cul']>=100,'1','2')
Acad['EMPPERC']= Acad['cul']
Acad['endt']= np.where(Acad['end_date'].astype(str).str.slice(0,4)=='9999', np.nan, Acad['end_date'])
Acad['endt']= (pd.to_datetime(Acad['endt']) - Acad['start_date'] ).dt.days
Acad['POSMO'] = np.where(Acad['endt'].isnull(), 12 ,
                        np.where(Acad['endt']/365*12>=12,12,
                              np.floor(Acad['endt']/365*12)  ))

Acad['EEO'] = Acad['eeo_cd'].str.slice(0,1)
#need to recheck coding for ethnic, records without racial_cat_cd_1 given white?
Acad['ETHNIC']= np.where(Acad['ethnc_cd']=='E1',3,
                        np.where(Acad['racial_cat_cd_1'] == 'R1',4,
                                np.where(Acad['racial_cat_cd_1']=='R2',7,
                                        np.where(Acad['racial_cat_cd_1']=='R3',1,
                                                np.where(Acad['racial_cat_cd_1']=='R4',6,
                                                        np.where(Acad['racial_cat_cd_1']=='R5',5, 5))))))

#no gndr_cd given Male by default?
Acad['GENDER']= np.where(Acad['gndr_cd']=='1','M',
                        np.where(Acad['gndr_cd']=='2','F','M'))

Acad['HANDICAP']= np.nan
Acad['CITIZEN']= np.where(Acad['res_status_cd']=='C','1',
                         np.where(Acad['res_status_cd']=='N','3',
                                 np.where(Acad['res_status_cd'].isin(['S','T','A']),'2','1')))

In [397]:
#code for degree
degS={'02':'GED',
'03':'SC',
'04':'AA',
'05':'AS',
'06':'ASC',
'07':'BA',
'08':'BS',
'10':'BFA',
'14':'BSN',
'15':'MA',
'20':'MS',
'22':'MAT',
'24':'MBA',
'26':'MED',
'28':'MFA',
'30':'MHA',
'32':'MLR',
'36':'MLS',
'39':'MN',
'40':'MPA',
'42':'MPH',
'46':'MSW',
'48':'DAG',
'50':'EDS',
'54':'DBA',
'58':'DDS',
'62':'EDD',
'64':'JD',
'66':'MD',
'82':'DO',
'86':'PHD',
'93':'DPA',
'98':'DVM'}

degF ={'07':'S0',
'08':'U0',
'10':'V0',
'12':'L0',
'15':'N0',
'20':'Y0',
'24':'M0',
'28':'H0',
'32':'R0',
'34':'F0',
'36':'W0',
'38':'Z0',
'44':'X0',
'46':'I0',
'50':'Q0',
'65':'J0',
'66':'B0',
'82':'C0',
'86':'A0',
'92':'K0',
'98':'D0',
'99':'P0'}

degS = pd.Series(degS)
degS.name ='DEGREE'
degS.index.name='certcd'
degS= degS.reset_index()

degF = pd.Series(degF)
degF.name ='DEGREE'
degF.index.name='certcd'
degF= degF.reset_index()

In [398]:
Acad = Acad.join(degF.set_index('certcd'), on='cert_cd')


In [483]:
Acad['APPTBAS']= np.where(Acad['emp_sgrp_cd'].isin(['AN','AO','AQ']),'AN',
                         np.where(Acad['emp_sgrp_cd'].isin(['AP','AR']),'AY', np.nan ))

Acad['SAL_FTE'] = (Acad['anl_sal']/Acad['cul'])*100
Acad['RANKPC']= np.where(Acad['job_ttl'].str.contains('Post'), 'XPF0',
                        np.where( (Acad['emp_cat_cd'].str.startswith('FX') ) & ( ~ Acad['status_cd'].isin(['TENR','TPRO'])  ) & (Acad['derived_rank'].isnull()),'0000',
                                np.where((Acad['emp_cat_cd'].str.startswith('FM') ) & (~ Acad['status_cd'].isin(['TENR','TPRO']) ) & (Acad['derived_rank'].isnull()),'0000',
                                        np.where(Acad['derived_rank']==4, 'AA00',
                                                np.where( (Acad['status_cd'].isin(['TENR','TPRO'])) & (Acad['derived_rank'].isnull()),'AA00',
                                                        np.where(Acad['derived_rank']==3, 'BB00',
                                                                np.where(Acad['derived_rank']==2,'CC00',
                                                                        np.where(Acad['derived_rank']==1,'DD00',
                                                                                np.where(Acad['job_ttl'].str.contains('- Advisor'),'SSA0',
                                                                                        np.where(Acad['job_ttl'].str.contains('- Curric'),'SSC0',
                                                                                                np.where(Acad['job_ttl'].str.contains('- Outreach'),'SSP0',
                                                                                                        np.where(Acad['job_ttl'].str.contains('- Research'),'SSR0',
                                                                                                                np.where(Acad['job_ttl'].str.contains('- Teacher'),'SST0',
                                                                                                                        np.where(Acad['emp_cat_cd'].isin(['MT','MW']),'MU00',
                                                                                                                                np.where(Acad['emp_cat_cd'].isin(['FLF','FLC']),'LLD0',
                                                                                                                                        np.where(Acad['emp_cat_cd'].isin(['NT','NW']), 'NEC0',
                                                                                                                                                np.where(Acad['job_ttl'].str.contains('Coach'),'VJ00',
                                                                                                                                                np.where(Acad['job_ttl'].str.contains('Athlet'),'VJ00',
                                                                                                                                                np.where(Acad['job_ttl'].str.contains('Lectur'),'TL00',
                                                                                                                                                np.where(Acad['job_ttl'].str.contains('Intern-'),'Ul00',
                                                                                                                                                np.where(Acad['job_ttl'].str.contains('Resident-'),'UR00',
                                                                                                                                                np.where(Acad['job_ttl'].str.contains('Scholar-'),'UVSV',
                                                                                                                                                np.where(Acad['job_ttl'].str.contains('Assistant Instructor'),'TA00',
                                                                                                                                                np.where(Acad['job_ttl'].str.contains('Research Associate'),'URA0','YZ00'))))))))))))))))))))))))

Acad['RPNAME']= np.where( (Acad['status_cd'].isin(['TENR','TPRO']) ) & (Acad['derived_rank'].isnull()), 'Professor',
                        np.where(Acad['derived_rank']==4, 'Professor',
                                np.where(Acad['derived_rank']==3, 'Assoc Professor',
                                        np.where(Acad['derived_rank']==2,'Asst Professor',
                                                np.where(Acad['derived_rank']==1, 'Instructor', None)))))

Acad['TBDATE'] = np.where(Acad['status_cd'].isnull(), Acad['start_date'],
                         np.where(Acad['status_cd'].str.endswith('E'),Acad['start_date'], None ))

Acad['TEDATE'] = np.where(Acad['status_cd'].isnull(), Acad['prim_asgn_end_date'],
                         np.where(Acad['status_cd'].str.endswith('E'),Acad['prim_asgn_end_date'], None ))
Acad['Age']= (( Acad['cutdt']-Acad['birth_date']).dt.days)/365

#change newhire using action ZA
Acad['New_Hires_Act'] = np.where(Acad['NewHire']=='Y',1, 0)
Acad['New_Hires'] = np.where( (Acad['cont_svc_date']>= np.datetime64(str(int(hrexdt[0:4])-1)+'-10-01') ) & \
                             (Acad['cont_svc_date']<= np.datetime64(hrexdt[0:4]+ '-10-01') )  ,1,0 )

Acad['vstdt']= np.where(Acad['vstd_date'].astype(str).str.slice(0,4)=='9999', None, Acad['vstd_date'])
Acad['Retire_Elig'] = np.where( (pd.to_datetime(Acad['vstdt']) <= pd.to_datetime(str(int(hrexdt[0:4])+1)+'-07-01' ) ) & \
                              (Acad['vstdt'].notnull()) ,1,0 )
Acad['FixTerm_Exp']= np.where((Acad['prim_asgn_end_date'] <= pd.to_datetime(str(int(hrexdt[0:4])+1)+'-07-01' ) ) & \
                             (Acad['prim_asgn_end_date'] != pd.to_datetime('1900-01-01')),1,0)


Acad['AR1']= np.where(Acad['Age']<35,1,0)
Acad['AR2']= np.where( (Acad['Age']>=35) & (Acad['Age']<=39),1,0)
Acad['AR3']= np.where( (Acad['Age']>=40) & (Acad['Age']<=44),1,0)
Acad['AR4']= np.where( (Acad['Age']>=45) & (Acad['Age']<=49),1,0)
Acad['AR5']= np.where( (Acad['Age']>=50) & (Acad['Age']<=54),1,0)
Acad['AR6']= np.where( (Acad['Age']>=55) & (Acad['Age']<=59),1,0)
Acad['AR7']= np.where( (Acad['Age']>=60) & (Acad['Age']<=64),1,0)
Acad['AR8']= np.where(Acad['Age']>=65,1,0)

def racerd(s):
    if s=='R1':
        r='4'
    elif s=='R2':
        r='7'
    elif s=='R3':
        r='1'
    elif s=='R4':
        r='6'
    elif s=='R5':
        r='5'
    else:
        r=None
    return r

Acad['ETHNIC1']= Acad['racial_cat_cd_1'].apply(racerd)
Acad['ETHNIC2']= Acad['racial_cat_cd_2'].apply(racerd)
Acad['ETHNIC3']= Acad['racial_cat_cd_3'].apply(racerd)
Acad['ETHNIC4']= Acad['racial_cat_cd_4'].apply(racerd)
Acad['ETHNIC5']= Acad['racial_cat_cd_5'].apply(racerd)
Acad['Total']=1
Acad['Tenure_System'] = np.where(Acad['status_cd'].isin(['TENR','TPRO']),1,0)
Acad['Fixed_Term_Faculty'] = np.where((Acad['derived_rank'].notnull()) & (Acad['status_cd'].isnull()),1,\
                                     np.where((Acad['derived_rank'].notnull()) & (Acad['status_cd'].str.endswith('E')),1,0))

#whether status_cd='COTR' cs or ft
Acad['Continuing_Staff'] = np.where((Acad['status_cd'].notnull()) & ( Acad['status_cd'].str.endswith('E') == False) \
                                   & (~ Acad['status_cd'].isin(['TENR','TPRO'])),1,0 )

Acad['Fixed_Term_Staff'] = np.where((Acad['derived_rank'].isnull()) & (Acad['status_cd'].str.endswith('E')),1,
                                   np.where((Acad['derived_rank'].isnull()) & (Acad['status_cd'].isnull()) ,1,0))

Acad['Ranked_Faculty']= Acad['Tenure_System'] +Acad['Fixed_Term_Faculty']
Acad['Unranked_Faculty']= np.where(Acad['Ranked_Faculty']==1,0,1)

#BC fixed term restrict on emp_cat_cd in('FMM','FFF','FXE','FHF','FNF') exclude specialist with rank
Acad['Fixed_Term_Faculty_BC']= np.where((Acad['derived_rank'].notnull()) & \
                                        (Acad['emp_cat_cd'].isin(['FMM','FFF','FXE','FHF','FNF'])) & (~Acad['status_cd'].isin(['TENR','TPRO'])),1,0)

In [484]:
Acad.query('Fixed_Term_Faculty !=Fixed_Term_Faculty_BC ').query('Fixed_Term_Faculty_BC==1').\
loc[:,['pers_id','status_cd','derived_rank','DerivedRnk_BC','Continuing_Staff','Age','AR1','AR5']]


,pers_id,status_cd,derived_rank,DerivedRnk_BC,Continuing_Staff,Age,AR1,AR5
5237,00009373,COTR,3.0,NaN,1,50.232877,0,1
37167,00051263,COTR,3.0,NaN,1,57.027397,0,0


In [485]:
Acad.columns.tolist()
#Acad['endt']= np.where(Acad['end_date'].astype(str).str.slice(0,4)=='9999', np.nan, Acad['end_date'])
#pd.to_datetime(Acad['endt']).dtype
#Acad['end_date'].astype(str).head().str.slice(0,11)
#(Acad['endt']-Acad['start_date']).dt.days
#Acad.dtypes

['pers_nbr',
 'start_date',
 'end_date',
 'pos_nbr',
 'job_cd',
 'org_cd',
 'asgn_typ_cd',
 'asgn_typ_nm',
 'prim_asgn_end_date',
 'payrl_area_cd',
 'payrl_area_nm',
 'pers_area_cd',
 'pers_area_nm',
 'pers_sarea_cd',
 'pers_sarea_nm',
 'emp_grp_cd',
 'emp_grp_nm',
 'emp_sgrp_cd',
 'emp_sgrp_nm',
 'cntct_cd',
 'cntct_nm',
 'profsnl_nm',
 'pers_id',
 'job_ttl',
 'eeo_cd',
 'eeo_nm',
 'emp_cat_cd',
 'emp_cat_nm',
 'fac_rnk_cd',
 'ssn',
 'uuid',
 'fst_nm',
 'mid_nm',
 'lst_nm',
 'birth_date',
 'gndr_cd',
 'gndr_nm',
 'ethnc_cd',
 'racial_cat_cd_1',
 'racial_cat_cd_2',
 'racial_cat_cd_3',
 'racial_cat_cd_4',
 'racial_cat_cd_5',
 'racial_cat_cd_6',
 'racial_cat_cd_7',
 'racial_cat_cd_8',
 'racial_cat_cd_9',
 'racial_cat_cd_10',
 'res_status_cd',
 'res_status_nm',
 'cont_svc_date',
 'Active',
 'acum_fte_svc_mths',
 'vstd_date',
 'cert_cd',
 'pay_scale_typ_cd',
 'pay_scale_area_cd',
 'wage_typ_cd_2',
 'pymt_wage_typ_amt_2',
 'pay_scale_grp_cd',
 'pay_scale_lvl_cd',
 'capc_util_lvl',
 'anl_sal

In [ ]:
########################################################
#########following to get NonAcad FTE details################

In [547]:
NAcadFTE= EmpFTE.query('emp_cat_cd_1=="S" ')
NAcadFTE['Employee_Group'] = NAcadFTE['emp_cat_cd'].str.slice(1,2)
NAcadFTE['OCCupation_CODE'] = '0'
NAcadFTE['FTE_Total'] = NAcadFTE['FTE_Funding_Derived']

NAcadFTE['FTE_GF'] = np.where(NAcadFTE['fund'].str.startswith('MSG'),NAcadFTE['FTE_Total'],
                             np.where((NAcadFTE['fund']=='MSXC023271') & (NAcadFTE['org_cd'] != '10070386') , NAcadFTE['FTE_Total']*0.7,
                                     np.where((NAcadFTE['fund']=='MSXC023271') & (NAcadFTE['org_cd'] == '10070386'), NAcadFTE['FTE_Total']*0.5,0)))


NAcadFTE['FTE_ERF']= np.where(NAcadFTE['fund'].str.startswith('MSR'), NAcadFTE['FTE_Total'],0)
NAcadFTE['FTE_DES']= np.where( (NAcadFTE['fund'].str.startswith('MSD') ) & (NAcadFTE['fund'].str.startswith('MSDC')== False), NAcadFTE['FTE_Total'],0)

NAcadFTE['FTE_AUX'] = np.where(NAcadFTE['fund'].str.startswith('MSX')== False,0,
                             np.where((NAcadFTE['fund']=='MSXC023271') & (NAcadFTE['org_cd'] != '10070386') , NAcadFTE['FTE_Total']*0.3,
                                     np.where((NAcadFTE['fund']=='MSXC023271') & (NAcadFTE['org_cd'] == '10070386'), NAcadFTE['FTE_Total']*0.5,NAcadFTE['FTE_Total'])))

NAcadFTE['FTE_OTH']= np.where(NAcadFTE['fund'].str.startswith('MSP'),NAcadFTE['FTE_Total'],
                             np.where(NAcadFTE['fund'].str.startswith('MSL'),NAcadFTE['FTE_Total'],0 ))

NAcadFTE['FULLPART']= np.where(NAcadFTE['cul']>=90,'1','2')
NAcadFTE['TIMe_code']=NAcadFTE['FULLPART']+'2'
NAcadFTE['EEO_code'] = NAcadFTE['EEO_code'].str.slice(0,1)

NAcadFTE['FUND'] = np.where(NAcadFTE['fund'].str.startswith('MSG'),'1',
                           np.where(NAcadFTE['fund'].str.startswith('MSR'),'2',
                                   np.where( (NAcadFTE['fund'].str.startswith('MSD') ) & (NAcadFTE['fund'].str.startswith('MSDC')== False),'3',
                                           np.where(NAcadFTE['fund'].str.startswith('MSX'),'4',
                                                   np.where(NAcadFTE['fund'].str.startswith('MSA'),'9',
                                                           np.where(NAcadFTE['fund'].str.startswith('MSL'),'5','5'))))))

C:\Users\chendi4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\chendi4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\chendi4\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

KeyError: 'EEO_code'

In [545]:
NAcadFTE['FULLPART'].value_counts()
(NAcadFTE['cul']/100).sum()

7729.1005000000005

In [527]:
tst= pd.read_sql('select * from Non_Aggregated.dbo.NonAcad_FTE where CYear=2011', OPBDB2)


In [528]:
tst=tst.groupby(['Pers_ID','CUC','Account'])['FTE_Total','FTE_GF'].sum()


In [530]:
b = NAcadFTE.loc[:,['mau_code_pay','dept_code_pay','pers_id','fund','org_cd_pay']]
b['CUC']= b['mau_code_pay']+b['dept_code_pay']
tst = b.join(tst, on=['pers_id','CUC','fund'] )

In [531]:
tst.query('FTE_Total != FTE_Total').head()
#.loc[:,['Pers_ID','CUC','mau_code_pay','dept_code_pay']].head()

,mau_code_pay,dept_code_pay,pers_id,fund,org_cd_pay,CUC,FTE_Total,FTE_GF
56438,10,117,00007352,MSGA011891,10010117,10117,NaN,NaN
73120,10,117,00027317,MSGA011891,10010117,10117,NaN,NaN
73120,10,117,00027317,MSRN031143,10010117,10117,NaN,NaN
60331,10,117,00029111,MSGA011891,10010117,10117,NaN,NaN
60497,10,117,00044877,MSGA011891,10010117,10117,NaN,NaN


In [514]:
pd.options.display.max_columns = None
display(NAcadFTE.query('pers_id=="00045158"'))

,pers_nbr,start_date,end_date,pos_nbr,job_cd,org_cd,asgn_typ_cd,asgn_typ_nm,prim_asgn_end_date,payrl_area_cd,payrl_area_nm,pers_area_cd,pers_area_nm,pers_sarea_cd,pers_sarea_nm,emp_grp_cd,emp_grp_nm,emp_sgrp_cd,emp_sgrp_nm,cntct_cd,cntct_nm,profsnl_nm,pers_id,job_ttl,eeo_cd,eeo_nm,emp_cat_cd,emp_cat_nm,fac_rnk_cd,ssn,uuid,fst_nm,mid_nm,lst_nm,birth_date,gndr_cd,gndr_nm,ethnc_cd,racial_cat_cd_1,racial_cat_cd_2,racial_cat_cd_3,racial_cat_cd_4,racial_cat_cd_5,racial_cat_cd_6,racial_cat_cd_7,racial_cat_cd_8,racial_cat_cd_9,racial_cat_cd_10,res_status_cd,res_status_nm,cont_svc_date,Active,acum_fte_svc_mths,vstd_date,cert_cd,pay_scale_typ_cd,pay_scale_area_cd,wage_typ_cd_2,pymt_wage_typ_amt_2,pay_scale_grp_cd,pay_scale_lvl_cd,capc_util_lvl,anl_sal,pos_ttl,status_cd,status_nm,tenure_cont_grant_dept,tenure_cont_status_grant_date,tenure_exmp_agr_date,tenure_cont_sys_entr_date,promo_to_cur_rnk_entr_date,NewHire,cutdt,emp_cat_cd_1,cul,DerivedRnk,DerivedRnk_BC,org_full_name,mau_code,mau_name,dept_code,dept_name,derived_rank,org_cd_pay,fund_grp_cd,fund,wt_pct,pymt_wage_typ_amt,sum_pymt_wage_typ_amt,org_full_name_pay,mau_code_pay,mau_name_pay,dept_code_pay,dept_name_pay,Weight_Percent,FTE_Funding_Derived,Statc,Employee_category,egrp,GFFTE,Employee_Group
60499,45158,2011-08-07,9999-12-31 00:00:00,30043766,20001767,10010117,None,None,1900-01-01,M1,MSU: Monthly,MSU1,MSU - US,APSA,Pro Supervisory,1,Union,A3,Salary - Level 1,None,None,Mark E Bailey,00045158,Administrative Assistant II/S,A2,Professionals,SS,SUPERVISORY,NaN,384722149,119C610B-9C6D-102A-AA45-59A83C375D2F,Mark,E,Bailey,1957-10-21,1,Male,E2,R5,None,None,None,None,None,None,None,None,None,C,Citizen,1984-11-26,Y,316.0,2009-11-01 00:00:00,06,01,01,None,0.0,APA01,11,100.0,50906.04,Administrative Assistant II/S,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,2011-10-01,S,100.0,NaN,NaN,BROADCASTING SERVICES,10,COLLEGE OF COMMUNICATION ARTS & SCIENCES,117,BROADCASTING SERVICES,NaN,10010117,GF,MSGA011891,100.0,4242.17,4242.17,BROADCASTING SERVICES,10,COLLEGE OF COMMUNICATION ARTS & SCIENCES,117,BROADCASTING SERVICES,100.0,1.0,TS,Non Acad,fy,1.0,S


In [494]:
NAcadFTE.columns

Index(['pers_nbr', 'start_date', 'end_date', 'pos_nbr', 'job_cd', 'org_cd',
       'asgn_typ_cd', 'asgn_typ_nm', 'prim_asgn_end_date', 'payrl_area_cd',
       'payrl_area_nm', 'pers_area_cd', 'pers_area_nm', 'pers_sarea_cd',
       'pers_sarea_nm', 'emp_grp_cd', 'emp_grp_nm', 'emp_sgrp_cd',
       'emp_sgrp_nm', 'cntct_cd', 'cntct_nm', 'profsnl_nm', 'pers_id',
       'job_ttl', 'eeo_cd', 'eeo_nm', 'emp_cat_cd', 'emp_cat_nm', 'fac_rnk_cd',
       'ssn', 'uuid', 'fst_nm', 'mid_nm', 'lst_nm', 'birth_date', 'gndr_cd',
       'gndr_nm', 'ethnc_cd', 'racial_cat_cd_1', 'racial_cat_cd_2',
       'racial_cat_cd_3', 'racial_cat_cd_4', 'racial_cat_cd_5',
       'racial_cat_cd_6', 'racial_cat_cd_7', 'racial_cat_cd_8',
       'racial_cat_cd_9', 'racial_cat_cd_10', 'res_status_cd', 'res_status_nm',
       'cont_svc_date', 'Active', 'acum_fte_svc_mths', 'vstd_date', 'cert_cd',
       'pay_scale_typ_cd', 'pay_scale_area_cd', 'wage_typ_cd_2',
       'pymt_wage_typ_amt_2', 'pay_scale_grp_cd', 'pay_scale

In [ ]:
import re
orgcol =[x for x in ASGN.columns if re.match(r'prim_asgn_also_rpt_to_\d',x) or re.match(r'\w+_org_cd_\d+',x) or \
 x in ['pers_nbr','start_date','end_date'] ]
orgdf = ASGN.filter(items=orgcol)
orgdfL = orgdf.melt(id_vars=['pers_nbr','start_date','end_date'], var_name='type', value_name='org_cd')
orgdfL = orgdfL.dropna()
orgdfL['seq']= orgdfL['type'].str.extract(r'\w+_(\d)$')
orgdfL['asgngrp'] = orgdfL['type'].str.extract(r'(\w+)_\d$')

orgdfL = orgdfL.join(STRUCT.set_index('org_code').loc[:,['mau','dept','org_full_name']], on='org_cd')

In [ ]:
EmpFTE.query('FTE_Funding_Derived>1')

In [ ]:
EmpFTE.columns

In [ ]:
pd.read_sql("select * from HR_"+hrexdt+".HR_OM_JOB_V \
                   where '"+dtfilter+"' between start_date and end_date", EDW).columns

In [ ]:
pd.read_sql("select * from HR_201210.HR_OM_JOB_V \
                   where '"+dtfilter+"' between start_date and end_date", EDW).columns

In [491]:
pd.read_sql("select * from HR_"+hrexdt+".HR_PA_PERSON_SV \
                   where '"+dtfilter+"' between start_date and end_date", EDW).head()

,pers_id,start_date,end_date,ssn,apid,zpid,uuid,oth_id,nm_ttl,fst_nm,...,gndr_nm,nm_fmt_cd,nm_fmt_nm,curr_ind,src_last_updated_by,src_last_update_date,created_by,create_date,last_updated_by,last_update_date
0,00008887,1900-01-01,9999-12-31 00:00:00,320865869,A39876796,None,13F6CD12-9C6D-102A-AA45-59A83C375D2F,None,None,Cory,...,Male,0,"First Name,First Name,Middle Name,Middle Name,...",Y,EBSP0001,2010-12-29,HR_PA_PERSON_SAP_EXTRACT,2011-04-01 16:51:11,HR_PA_PERSON_SAP_EXTRACT,2011-10-24 21:20:56
1,00008889,1978-12-06,9999-12-31 00:00:00,320981327,None,Z41141739,5F807E7A-D527-102B-878B-3071A4E281DE,None,None,Mohamed,...,Male,0,"First Name,First Name,Middle Name,Middle Name,...",Y,EBSP0001,2010-12-29,HR_PA_PERSON_SAP_EXTRACT,2011-04-01 16:51:11,HR_PA_PERSON_SAP_EXTRACT,2011-10-24 21:20:56
2,00008890,1914-08-26,9999-12-31 00:00:00,321058576,None,Z41057806,140C4A4D-9C6D-102A-AA45-59A83C375D2F,None,None,Eunice,...,Female,0,"First Name,First Name,Middle Name,Middle Name,...",Y,EBSP0001,2010-12-29,HR_PA_PERSON_SAP_EXTRACT,2011-04-01 16:51:11,HR_PA_PERSON_SAP_EXTRACT,2011-10-24 21:20:56
3,00008891,1977-04-17,9999-12-31 00:00:00,321068021,None,Z41210736,2A1D1B06-78CE-102D-AF24-495BEE9521E4,None,None,Adarsh,...,Male,0,"First Name,First Name,Middle Name,Middle Name,...",Y,EBSP0001,2010-12-29,HR_PA_PERSON_SAP_EXTRACT,2011-04-01 16:51:11,HR_PA_PERSON_SAP_EXTRACT,2011-10-24 21:20:56
4,00008892,1937-11-23,9999-12-31 00:00:00,321300646,None,Z40339299,13BACE9A-9C6D-102A-AA45-59A83C375D2F,None,None,Janice,...,Female,0,"First Name,First Name,Middle Name,Middle Name,...",Y,EBSP0001,2010-12-29,HR_PA_PERSON_SAP_EXTRACT,2011-04-01 16:51:11,HR_PA_PERSON_SAP_EXTRACT,2011-10-24 21:20:56
